Examples for `dicomo.py` subpackage, (Co)-Moment Estimates
====================================================
To run a toy example, start by sourcing packages and data: 

In [1]:
# Load data
import pandas as ps
import numpy as np
import scipy.stats as sps
import statsmodels.robust as srs
from direpack import dicomo
import dcor as dc
data = ps.read_csv("../data/Returns_shares.csv")
columns = data.columns[2:8]
(n,p) = data.shape
datav = np.matrix(data.values[:,2:8].astype('float64'))
y = datav[:,0]
x = datav[:,1]

ModuleNotFoundError: No module named 'cyipopt'

Product-Moment Statistics
=========================
a) Moments
----------
Let's compare some moment estimates to `numpy`'s version. 

 - Second moment: variance 

In [2]:
# Variance 
covest = dicomo() 
# division by n
print("Variance without finite sample correction estimated by dicomo:" + str(covest.fit(x,biascorr=False)))
print("Variance without finite sample correction estimated by numpy:" + str(np.var(x)))
        
# division by n-1 
print("Variance with finite sample correction estimated by dicomo:" + str(covest.fit(x,biascorr=True)))
print("Variance with finite sample correction estimated by numpy:" + str(np.var(x)*n/(n-1)))
        
    
# Robust trimmed variance: 
print("Robust 10% trimmed variance:" + str(covest.fit(x,biascorr=False,trimming=.1)))
        
# Nonparametric Scale: MAD
# NB at this point, this is the only exception where dicomo will yield scale instead of the moment itself
covest.set_params(center='median')
print("MAD ex dicomo:" + str(covest.fit(x)))
print("MAD ex statsmodels:" + str(srs.mad(x)))


NameError: name 'dicomo' is not defined

- third moment and skewness: 

In [4]:
# Third Moment  
# division by n
covest.set_params(center='mean',mode='mom')
print(covest.get_params())
print("Third moment estimated by dicomo:" + str(covest.fit(x,biascorr=False,order=3)))
print("Third moment estimated by scipy.stats:" + str(sps.moment(x,3)))
        
# skewness 
covest.set_params(mode='skew')
print("Skewness estimated by dicomo without small sample correction:" + str(covest.fit(x,biascorr=False)))
print("Skewness estimated by scipy without small sample correction:" + str(sps.skew(x)))
print("Skewness estimated by dicomo with small sample correction:" + str(covest.fit(x,biascorr=True)))
print("Skewness estimated by scipy with small sample correction:" + str(sps.skew(x,bias=False)))

{'center': 'mean', 'est': 'arithmetic', 'mode': 'mom'}
Third moment estimated by dicomo:1.2796570257022306
Third moment estimated by scipy.stats:[1.27965703]
Skewness estimated by dicomo without small sample correction:0.3881195552213812
Skewness estimated by scipy without small sample correction:[0.38811956]
Skewness estimated by dicomo with small sample correction:0.3882872295778524
Skewness estimated by scipy with small sample correction:[0.38828723]


- Fourth Moment and Kurtosis

In [5]:
# 4th Moment 
covest.set_params(mode='mom')
print("Fourth moment estimated by dicomo:" + str(covest.fit(x,biascorr=False,order=4)))
print("Fourth moment estimated by scipy.stats:" + str(sps.moment(x,4)))

# Again, we can trim: 
print("Fourth moment estimated by dicomo, 20% trimmed:" + str(covest.fit(x,biascorr=False,order=4,trimming=.2)))

#Kurtosis 
covest.set_params(mode='kurt')
print("Kurtosis estimated by dicomo without any corrections:" + str(covest.fit(x,biascorr=False)))
print("Kurtosis estimated by scipy without any correction:" + str(sps.kurtosis(x,fisher=False,bias=True)))
print("Kurtosis estimated by dicomo with small sample correction:" + str(covest.fit(x,biascorr=True,Fisher=False)))
print("Kurtosis estimated by scipy with small sample correction:" + str(sps.kurtosis(x,fisher=False,bias=False)))
print("Kurtosis estimated by dicomo with small sample and Fisher corrections:" + str(covest.fit(x,biascorr=True,Fisher=True)))
print("Kurtosis estimated by scipy with small sample and Fisher corrections:" + str(sps.kurtosis(x,fisher=True,bias=False)))

Fourth moment estimated by dicomo:88.13900011202283
Fourth moment estimated by scipy.stats:[88.13900011]
Fourth moment estimated by dicomo, 20% trimmed:0.846793760134836
Kurtosis estimated by dicomo without any corrections:17.961007966358487
Kurtosis estimated by scipy without any correction:[17.96100797]
Kurtosis estimated by dicomo with small sample correction:17.984292234604126
Kurtosis estimated by scipy with small sample correction:[17.98429223]
Kurtosis estimated by dicomo with small sample and Fisher corrections:14.984292234604126
Kurtosis estimated by scipy with small sample and Fisher corrections:[14.98429223]


b) Co-Momemnts 
--------------
- Second order co-moment

In [6]:
# Covariance 
covest.set_params(mode='com')
print("Covariance estimated by pandas:" + "\n" + str(data.iloc[:,2:4].cov()))
print("Covariance estimated by dicomo:" + str(covest.fit(x,y=y,biascorr=True)))

# Trimmed Covariance
print("10% Trimmed Covariance estimated by dicomo:" + str(covest.fit(x,y=y,biascorr=True,trimming=.1)))

Covariance estimated by pandas:
          KMB       XOM
KMB  1.147585  0.748737
XOM  0.748737  2.215867
Covariance estimated by dicomo:0.7487368016330135
10% Trimmed Covariance estimated by dicomo:0.2737112962842033


Note that `pandas` calculates covariance with small sample correction by default (division by n - 1)

- Third order co-moments

In [7]:
# Third order co-moment (x,x,y) 
covest.set_params(mode='com')
print("Third co-moment estimated by dicomo:" + str(covest.fit(x,y=y,biascorr=True,option=1,order=3)))
# Co-skewness (x,x,y)
covest.set_params(mode='cos')
print("Co-skewness estimated by dicomo:" + str(covest.fit(x,y=y,biascorr=True,option=1)))

Third co-moment estimated by dicomo:0.3900937277965764
Co-skewness estimated by dicomo:0.1643360470693657


Note that is is difficult to find a benchmark to compare against. Some higher order co-moments can be found in the `R` package `PerformanceAnalytics`. A comparison to those is beyond the scope of this notebook.

- Fourth order co-moments 

By now, we get how the class works.

In [8]:
# Co-Kurtosis
covest.set_params(mode='cok')
print(covest.get_params())
print("Co-kurtosis estimated integrally by dicomo:" + str(covest.fit(x,y=y,biascorr=False,option=1)))





{'center': 'mean', 'est': 'arithmetic', 'mode': 'cok'}
Co-kurtosis estimated integrally by dicomo:10.616246070495807


- Correlation

In [9]:
# Correlation 
covest.set_params(mode='corr')
print("Pearson correlation matrix estimated by pandas:" + "\n" + str(data.iloc[:,2:4].corr()))
print("Pearson correlation coefficient estimated by dicomo:" + str(covest.fit(x,y=y)))

# Will also do a trimmed version
print("10% trimmed Pearson correlation coefficient estimated by dicomo:" + str(covest.fit(x,y=y,trimming=.1)))

Pearson correlation matrix estimated by pandas:
          KMB       XOM
KMB  1.000000  0.469532
XOM  0.469532  1.000000
Pearson correlation coefficient estimated by dicomo:0.46953169893364827
10% trimmed Pearson correlation coefficient estimated by dicomo:0.3812162776658803


- Continuum Association

The continuum option provides any estimate in the continuum from variance to correlation coefficient. The trimmed version of this option is the projection index in Robust Continuum Regression (RCR) \[1\]. Setting `alpha = 1` yeilds covariance; the higher alpha, the closer it will get to variance. Setting alpha closer to zero lets it approach Pearson correlation.  

In [10]:
# Continuum 
covest.set_params(mode='continuum')
print(covest.get_params())
print("Covariance matrix estimated by pandas:" + "\n" + str(data.iloc[:,2:4].cov()))
print("Covariance estimated in dicomo continuum:" + str(np.sqrt(covest.fit(x,y=y,alpha=1,biascorr=True))))
print("Robust 10% trimmed continuum association:" + str(np.sqrt(covest.fit(x,y=y,alpha=.5,biascorr=True,trimming=.1))))

{'center': 'mean', 'est': 'arithmetic', 'mode': 'continuum'}
Covariance matrix estimated by pandas:
          KMB       XOM
KMB  1.147585  0.748737
XOM  0.748737  2.215867
Covariance estimated in dicomo continuum:0.7487368016330135
Robust 10% trimmed continuum association:0.27549572197011707


Energy Statistics
=================
- distance variance: 

In [12]:
# Variance
n=len(x)
covest.set_params(est='distance',mode='var')
print("Distance Statistics estimated by dcov" + "\n" +  str(dc.distance_stats(x,x).covariance_xy))
print("Distance variance estimated by dicomo:" + str(covest.fit(x,biascorr=False)))
print("Distance Statistics, unibiased in high dimension, estimated by dcov" + "\n" +  str(np.sqrt(dc.u_distance_stats_sqr(x,x).covariance_xy)))
print("Distance variance, unibiased in high dimension, estimated by dicomo:" + str(covest.fit(x,biascorr=True)))
print("Distance variance, unibiased in high dimension, estimated by dicomo, naive calculation mode:" + "\n" + str(covest.fit(x,biascorr=True,calcmode='slow')))
# The default for var, cov is to run 'fast' calculation mode where possible

Distance Statistics estimated by dcov
0.7484726076018628
Distance variance estimated by dicomo:0.7484726076018628
Distance Statistics, unibiased in high dimension, estimated by dcov
0.7477359326096397
Distance variance, unibiased in high dimension, estimated by dicomo:0.7477359326096397
Distance variance, unibiased in high dimension, estimated by dicomo, naive calculation mode:
0.7477359326096309


Note that, just like for the product-moment statistics, there is an option to use different internal centring.
Medians can be plugged in by setting `center='median'` and trimmed means can be plugged in by passing a trimming fraction. However, for energy statistics, it is unclear what the properties of the resulting estimate will be. 

In [13]:
covest.fit(x,biascorr=False,trimming=.1)

0.5663926271002548

- distance covariance:

In [14]:
covest.set_params(mode='com')
print("Distance Covariance estimated by dcov: " +  str(dc.distance_covariance(x,y)))
print("Distance Covariance estimated by dicomo: " + str(covest.fit(x,y=y,biascorr=False)))
print("Distance Covariance estimated by dicomo, naive calculation mode: " + str(covest.fit(x,y=y,biascorr=False,calcmode='slow')))
# The versions unbiased in high dimension, can be assessed as above

Distance Covariance estimated by dcov: 0.24601858995627712
Distance Covariance estimated by dicomo: 0.24601858995627712
Distance Covariance estimated by dicomo, naive calculation mode: 0.24601858995627846


- martingale difference divergence

In [15]:
covest.set_params(mode='mdd')
print("Martingale difference divergence estimated by dicomo: " +  str(covest.fit(x,y=y)))

Martingale difference divergence estimated by dicomo: 0.3524271500865358


- distance correlation

In [16]:
covest.set_params(mode='corr')
print("Distance Correlation estimated by dcov: " +  str(dc.distance_correlation(x,y)))
print("Distance Correlation estimated by dicomo: " + str(covest.fit(x,y=y)))
print("Distance Correlation estimated by dicomo, naive calculation mode: " + str(covest.fit(x,y=y,calcmode='slow')))

Distance Correlation estimated by dcov: 0.37849170732225507
Distance Correlation estimated by dicomo: 0.37849170732225507
Distance Correlation estimated by dicomo, naive calculation mode: 0.3784917073222609


- martingale difference correlation: 

In [17]:
covest.set_params(mode='mdc')
print("Martingale difference correlation estimated by dicomo: " +  str(covest.fit(x,y=y)))

Martingale difference correlation estimated by dicomo: 0.44837499675866715


- distance continuum association: 

In [18]:
covest.set_params(mode='continuum')
print("Distance Covariance estimated by dcov: " +  str(dc.distance_covariance(x,y)))
print("Distance Covariance estimated through continuum: " + str(np.sqrt(covest.fit(x,y=y,alpha=1,biascorr=False)))) 
print("Distance Continuum Association with small alpha: " + str(np.sqrt(covest.fit(x,y=y,alpha=.22,biascorr=False))))

Distance Covariance estimated by dcov: 0.24601858995627712
Distance Covariance estimated through continuum: 0.24601858995627712
Distance Continuum Association with small alpha: 0.2603176544258358
